# Network analysis in Senegal

### Objectives
    1)	Use measures of road-based accessibility to identify road segments that, if rehabilitated, would improve agricultural market activities in Senegal, including during flood conditions.
    2)	Gain a better understanding of the accessibility, connectivity, and criticality of roads in Senegal in relationship to agricultural origins, processing & transfer sites, and markets.

To this end, the team will develop an accessibility model which measures the travel time from sites of agricultural production to their nearest populated areas, processing centers, and markets. 

### Datasets for analysis
#### ORIGIN
    1) agriculture: MapSPAM 2017. Measuring value in international dollars.
    2) agriculture: UMD Land Cover 2019 30m. Assign MapSPAM value onto land cover cropland class for more precise origin information.
    3) population: WorldPop 2020, UN-adjusted.
    4) settlement extent: GRID3 2020.
#### DESTINATION
    4) markets: derived from WorldPop 2020 and GRID3 2020 urban clusters.
    5) agricultural processing hubs: to be acquired.
#### TRAVEL ROUTE
    6) roads: OpenStreetMap, July 2021.
    7) elevation: 
#### OBSTACLE
    8) flood: FATHOM. 1-in-10, 20, and 50 year flood return periods. 
#### INTERVENTION
    9) upcoming road projects: AGEROUTE interventions separate from the World Bank-financed project
    10) targeted road projects: critical road segments identified by this accessibility model's baseline outputs


### Model design
#### Basic formula: 
    (a) Off-road driving time from origin to closest road node
    +
    (b) Driving time from road node in (a) to a destination (closeness measured by road segments speeds)

#### Model origin & destination (OD) sets:
    A)	Travel time from an area that has agricultural value/potential to the nearest processing hub (if provided).
    B)	Travel time from an area that has agricultural value/potential to the nearest larger settlement, (“larger” settlement identified using a case-appropriate population metric to be determined).
    C)	Travel time from an area that has agricultural value/potential to the nearest market.
    D)	Travel time from all settlements to the nearest market.
    E)	Travel time from larger settlements to the nearest market.

#### Before/after scenarios for each OD set:
    1)	Pre-project, baseline weather: No inclement weather. Road network status as of November 2021.
    2)	Pre-project, flood: 1-in-10, 1-in-20 and 1-in-50 year flood return period. Road network status as of November 2021.
    3)	Post-project, baseline weather: No inclement weather. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).
    4)	Post-project, flood: 1-in-10 year flood return period. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).

#### Notes:
    --Destinations are expected to be proximal to the road network, so no measure is taken between road and destination.
    --All travel times will be assigned to each model variation’s point of origin; the aggregation up to admin areas is possible if desired.
    --Obstacles & interventions modify the road segment speeds. Basic formula is then applied to the modified road network.


### Prep workspace

In [1]:
import os, sys
GISFolder = os.getcwd()
GISFolder

'C:\\Users\\wb527163\\GEO-Cdrive-Grace'

In [2]:
# Note: needed to reinstall rtree due to geopandas import error. Did so in the console. 
# conda install -c conda-forge rtree=0.9.3

In [3]:
# load and filter osm network (step 1)
import geopandas as gpd
from geopandas import GeoDataFrame
import pandas as pd
import time
sys.path.append(r"C:\Users\wb527163\.conda\envs\geo\GOSTnets-master")
import GOSTnets as gn

In [4]:
import networkx as nx
import osmnx as ox
import numpy as np
import rasterio as rt
import shapely
from shapely.geometry import Point, box
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely import wkt
from shapely.geometry import LineString, MultiLineString, Point
import peartree

In [5]:
#### Might not use these
import fiona
from osgeo import gdal
import importlib
import matplotlib.pyplot as plt
import subprocess, glob

In [6]:
pth = os.path.join(GISFolder, "SEN-Cdrive") # Personal folder system for running model.
pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive'

In [7]:
out_pth = os.path.join(GISFolder, "SEN-Cdrive\outputs") # For storing intermediate outputs from the model.
out_pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive\\outputs'

In [8]:
team_pth = 'R:\\SEN\\GEO' # This is where the unmodified input data is stored. Finalized outputs also housed here.
team_pth

'R:\\SEN\\GEO'

### Prepare and clean the data

#### Return periods from FATHOM: 1-in-10, 20, and 50 year floods.

In [1]:
import multiprocessing

multiprocessing.cpu_count()

8

In [2]:
n_jobs=7

In [ ]:
flood = os.path.join(pth, "FU_1in10.csv")
flood = pd.read_csv(flood)
flood.info()

In [ ]:
crs = "EPSG:4326"
geometry = [Point(xy) for xy in zip(flood.x, flood.y)]
flood = GeoDataFrame(flood, crs=crs, geometry=geometry)
flood = gpd.read_file(flood)
flood

In [ ]:
flood = flood[['ID', 'FLD_ZONE', 'R10', 'R20', 'R50','geometry']]

In [ ]:
flood.crs

In [ ]:
gTime_edge = os.path.join(out_pth, "drive_time_edge.csv")
gTime_edge = pd.read_csv(gTime_edge)
crs = 'epsg:4326'
gTime_edge = GeoDataFrame(gTime_edge, crs=crs, geometry='geometry') 
gTime_edge

In [ ]:
gTime_edge.crs

In [ ]:
flood = flood.to_crs("EPSG:4326")
gTime_edge = gTime_edge.to_crs("EPSG:4326")
flood == gTime_edge

### Update driving times based on flood intersection.

#### Join road network and flood raster into single table.

In [ ]:
join = gpd.sjoin(gTime_edge, flood, how="left", lsuffix='_edge', rsuffix='_fl') # This takes 30+ minutes.
join.info()

In [ ]:
# Fewer errors farther down when using dataframe instead of gdf
gTime_flood = pd.DataFrame(join)
gTime_flood.info()

In [ ]:
gTime_flood.to_csv(os.path.join(out_pth, 'gTime_flood.csv'))

### Create speed penalties.

In [ ]:
gTime_flood.reset_index(inplace=True) # To avoid:  ValueError: cannot reindex from a duplicate axis.

In [ ]:
gTime_flood["time_R10"] = 0 # This is the "penalty" column
gTime_flood["time_R20"] = 0
gTime_flood["time_R50"] = 0

# 1-in-10 year flood
gTime_flood.loc[(gTime_flood['R10'] == 1), # Select rows where depth is 1 foot
       'time_R10'] = (gTime_flood['time'] * 1.25) # then slow down slightly on the penalty column
gTime_flood.loc[(gTime_flood['R10'] == 2), 
       'time_R10'] = (gTime_flood['time'] * 2) # Double the travel time on that road edge.
gTime_flood.loc[(gTime_flood['R10'] == 3), 
       'time_R10'] = (gTime_flood['time'] * 5) # Increase travel time five-fold.

# 1-in-20 year flood
gTime_flood.loc[(gTime_flood['R20'] == 1), # Select rows where depth is 1 foot
       'time_R20'] = (gTime_flood['time'] * 1.25) # then slow down slightly on the penalty column
gTime_flood.loc[(gTime_flood['R20'] == 2), 
       'time_R20'] = (gTime_flood['time'] * 2) # Double the travel time on that road edge.
gTime_flood.loc[(gTime_flood['R20'] == 3), 
       'time_R20'] = (gTime_flood['time'] * 5) # Increase travel time five-fold.

# 1-in-50 year flood
gTime_flood.loc[(gTime_flood['R50'] == 1), # Select rows where depth is 1 foot
       'time_R50'] = (gTime_flood['time'] * 1.25) # then slow down slightly on the penalty column
gTime_flood.loc[(gTime_flood['R50'] == 2), 
       'time_R50'] = (gTime_flood['time'] * 2) # Double the travel time on that road edge.
gTime_flood.loc[(gTime_flood['R50'] == 3), 
       'time_R50'] = (gTime_flood['time'] * 5) # Increase travel time five-fold.

gTime_flood

In [ ]:
gTime_flood.to_csv(os.path.join(out_pth, 'gTime_flood.csv'))

### Convert back to graph object.

In [ ]:
gTime_flood_G = nx.from_pandas_edgelist(
    gTime_flood,
    source="stnode",
    target="endnode",
    edge_attr=True,
)
# This should only take a few seconds.

gn.example_edge(gTime_flood_G, 10)

In [ ]:
gn.save(gTime_flood_G, 'gTime_flood', pth, edges = False, nodes = False)

### Origins and destinations

Measure distance from origin/destination to nearest node and save to file.

In [36]:
#%% If starting new session, reload graph from file
gTime = nx.read_gpickle("SEN-Cdrive/outputs/gTime.pickle")
agriculture = gpd.read_file("SEN-Cdrive/outputs/agriculture.shp")
agriculture = agriculture.to_crs("EPSG:4326")
agriculture.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 319 entries, 0 to 318
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Unnamed_ 0  319 non-null    int64   
 1   mgrs_code   319 non-null    object  
 2   type        319 non-null    object  
 3   AREA_GEO    319 non-null    float64 
 4   SUM         318 non-null    float64 
 5   Total_Driv  312 non-null    float64 
 6   WP_dens     319 non-null    float64 
 7   urb         319 non-null    int64   
 8   hd_urb      319 non-null    int64   
 9   Urb_class   319 non-null    int64   
 10  Shape_Leng  319 non-null    float64 
 11  Shape_Area  319 non-null    float64 
 12  geometry    319 non-null    geometry
dtypes: float64(6), geometry(1), int64(4), object(2)
memory usage: 32.5+ KB


In [37]:
agriculture.dtypes

Unnamed_ 0       int64
mgrs_code       object
type            object
AREA_GEO       float64
SUM            float64
Total_Driv     float64
WP_dens        float64
urb              int64
hd_urb           int64
Urb_class        int64
Shape_Leng     float64
Shape_Area     float64
geometry      geometry
dtype: object

In [39]:
?gn.pandana_snap_c

In [38]:
ag_snap = gn.pandana_snap_c(gTime, agriculture, source_crs = 'epsg:4326', target_crs = 'epsg:31028', add_dist_to_node_col = True)
ag_snap.to_csv('SEN-Cdrive/outputs/ag_snap.csv', index=True)
# Finished within a minute.
ag_snap

,Unnamed_ 0,mgrs_code,type,AREA_GEO,SUM,Total_Driv,WP_dens,urb,hd_urb,Urb_class,Shape_Leng,Shape_Area,geometry,NN,NN_dist
0,0,28PCU1268_02,bua,6.825291,14992.022066,67.224775,2196.539770,1,0,1,0.430449,0.000567,POINT (-16.72670 12.37174),2283044923,84.109763
1,1,28PCU6189_01,bua,27.151434,186046.609754,0.000000,6852.183502,1,1,2,0.671746,0.002259,POINT (-16.27319 12.56237),2119246531,39.014876
2,2,28PCV6615_01,bua,12.946680,29013.022570,26.304661,2240.962297,1,0,1,0.349253,0.001078,POINT (-16.22947 12.79722),1894405285,22.797121
3,3,28PDV3804_01,bua,7.396509,31580.806516,90.292621,4269.691037,1,0,1,0.252963,0.000616,POINT (-15.56341 12.70399),3355695913,42.169725
4,4,28PHU0490_01,bua,12.646372,36115.813159,158.705391,2855.824066,1,0,1,0.399690,0.001052,POINT (-12.19376 12.56342),8597133978,13.243242


### Create travel time values for the road nodes nearest to each service.

Using calculate_OD.

In [44]:
# If starting a new session, load from file.
HDurban_snap = os.path.join(out_pth, "HDurban_snap.csv")
HDurban_snap = pd.read_csv(HDurban_snap)
ag_snap = os.path.join(out_pth, "ag_snap.csv")
ag_snap = pd.read_csv(ag_snap)
gTime = nx.read_gpickle("SEN-Cdrive/outputs/gTime.pickle")

In [45]:
# We only need to find the origin-destination pairs for nodes closest to the origins and services,
# and some nodes will be the nearest for more than one service (and definitely for multiple origins).
origins = list(ag_snap.NN.unique())

In [46]:
list_HDurban = list(HDurban_snap.NN.unique()) 
dests = list_HDurban

# If more than one set of destinations, you can combine the sets to go into a single OD matrix using this code:
# list_ssa = list(ssa_snap.NN.unique()) 
# destslist = list_HDurban + list_ssa
# dests = list(set(destslist))

In [47]:
len(origins) # 23108 unique nearest nodes (average of 5 origins per node).

23108

In [48]:
len(dests) # 314 unique nearest nodes. This is only 5 less than the original dataset.

314

In [49]:
fail_value = 999999999 # If there is no shortest path, the OD pair will be assigned the fail value.

In [50]:
gn.example_edge(gTime)

(358284990, 5329792501, {'osmid': 59618174, 'ref': 'D 523', 'name': 'D 523', 'highway': 'unclassified', 'oneway': False, 'length': 73.22200000000001, 'geometry': <shapely.geometry.linestring.LineString object at 0x00000181229D7E20>, 'time': 5.271984000000001, 'mode': 'drive'})


In [51]:
OD = gn.calculate_OD(gTime, origins, dests, fail_value, weight = 'time')
# This took several hours.

In [52]:
OD_df = pd.DataFrame(OD, index = origins, columns = dests)

In [53]:
OD_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23108 entries, 7761872879 to 4072041221
Columns: 314 entries, 2283044923 to 3306192793
dtypes: float64(314)
memory usage: 55.5 MB


In [54]:
OD_df.tail()

,2283044923,2119246531,1894405285,3355695913,8597133978,3490825773,8636590476,3495281736,6029312286,6968809270,...,6045379010,6457408831,3783356554,7037053972,6045711032,1863721184,3649063162,293471544,3916091448,3306192793
8449983186,29330.831943,26016.463756,24504.783547,23510.273943,35485.967587,27506.451031,21183.588247,27267.712649,24923.205269,28901.469599,...,4268.289360,12736.435866,16259.315946,16113.610950,3447.737811,9591.049401,11524.398621,27189.837337,10178.882422,18612.700785
4072935008,29658.449919,26344.081732,24832.401523,23837.891919,35813.585563,27834.069007,21511.206223,27595.330625,25250.823245,29229.087575,...,6548.458191,11861.002788,15383.882868,15238.177872,2693.532750,8715.616323,10648.965543,27517.455313,10506.500398,17737.267707
4072935012,29692.485789,26378.117602,24866.437393,23871.927789,35847.621433,27868.104877,21545.242093,27629.366495,25284.859115,29263.123445,...,6582.494061,11860.194018,15383.074098,15237.369102,2692.723980,8714.807553,10648.156773,27551.491183,10540.536268,17736.458937
8558304561,30715.870059,27401.501872,25889.821663,24895.312059,36871.005703,28891.489147,22568.626363,28652.750765,26308.243385,30286.507715,...,7605.878331,12276.716589,15799.596669,15653.891673,3226.918722,9131.330124,11064.679344,28574.875453,11563.920538,18152.981508
4072041221,29729.441871,26415.073684,24903.393475,23908.883871,35884.577515,27905.060959,21582.198175,27666.322577,25321.815197,29300.079527,...,4541.957328,12879.657990,16402.538070,16256.833074,3590.959935,9734.271525,11667.620745,27588.447265,10577.492350,18755.922909


In [55]:
# Convert to minutes and save to file.
OD_min = OD_df[OD_df <fail_value] / 60
OD_min.to_csv(os.path.join(out_pth, 'fromagriculture/OD_flood.csv'))

In [56]:
OD_min.tail(20)

,2283044923,2119246531,1894405285,3355695913,8597133978,3490825773,8636590476,3495281736,6029312286,6968809270,...,6045379010,6457408831,3783356554,7037053972,6045711032,1863721184,3649063162,293471544,3916091448,3306192793
8073203019,360.914303,305.674833,280.480163,263.905003,492.809893,330.507954,225.126908,326.528981,287.453858,353.758264,...,134.132904,361.294538,330.176853,339.439415,206.482904,308.871430,341.093917,325.231059,80.687359,345.071244
8071408643,356.966502,301.727032,276.532362,259.957202,488.862092,326.560153,221.179107,322.581180,283.506057,349.810463,...,139.486675,366.648309,330.206267,339.468830,211.836675,314.225202,346.447689,321.283258,76.739558,345.100658
3717192287,355.289361,300.049891,274.855221,258.280061,457.874955,324.883012,219.501966,320.904039,281.828916,348.133322,...,123.714349,324.195482,284.806059,294.068622,185.533737,271.772374,303.994861,319.606118,36.090202,299.700450
4186362309,556.622422,500.260202,492.396391,437.041708,331.658413,542.424183,437.043136,330.256648,360.656559,302.678016,...,418.425640,177.003454,118.882474,128.145036,337.987056,229.426562,202.824827,475.507755,364.275842,106.254608
7805596740,399.906374,344.666904,319.472234,302.897074,531.801964,369.500026,264.118979,365.521052,326.445929,392.750335,...,124.049748,352.990054,366.786843,376.049406,198.178420,300.566946,332.789433,364.223131,124.191002,381.681234
4071912759,413.261679,358.022210,332.827540,316.252379,525.340593,382.855331,277.474285,378.876358,339.801235,406.105640,...,57.405258,284.566892,282.601937,291.864500,129.755258,232.143785,264.366272,377.578436,103.555840,297.496328
6021242885,413.522618,358.283148,333.088478,316.513318,525.079654,383.116269,277.735223,379.137296,340.062173,406.366579,...,57.144320,284.305954,282.340999,291.603561,129.494320,231.882846,264.105333,377.839375,103.294901,297.235390
6026792324,418.684576,363.445106,338.250436,321.675276,521.270170,388.278227,282.897181,384.299254,345.224131,411.528537,...,60.689634,283.910628,278.531514,287.794077,131.119460,231.487520,263.710007,383.001332,99.485417,293.425905
2281084786,416.216067,360.976597,335.781927,319.206767,518.801661,385.809719,280.428672,381.830746,342.755623,409.060028,...,62.062304,285.283297,276.063006,285.325568,132.492130,232.860189,265.082676,380.532824,97.016908,290.957397
9206374054,445.127973,389.888503,364.693833,348.118673,482.223569,414.721624,309.340578,410.742651,371.667528,407.857460,...,191.387696,220.472625,211.662694,220.925256,215.625927,168.049518,200.272005,409.444729,124.474588,226.557085


### Filter 1st nearest

#### Check each file to make sure nearest neighbor column is named correctly. If not, rename.

In [57]:
# Reload from file even if already loaded. Quickest way to ensure NN is a column rather than only the index.
OD = os.path.join(out_pth, "fromagriculture/OD_flood.csv")
OD = pd.read_csv(OD)

In [60]:
OD

,NN,2283044923,2119246531,1894405285,3355695913,8597133978,3490825773,8636590476,3495281736,6029312286,...,6045379010,6457408831,3783356554,7037053972,6045711032,1863721184,3649063162,293471544,3916091448,3306192793
0,7761872879,3.601602,62.713982,84.233782,178.427721,505.950746,146.401548,139.744338,238.725532,199.650409,...,474.253205,634.036094,577.000206,586.262769,542.843904,581.612986,613.835473,84.746116,324.588514,591.894597
1,7761872860,2.879157,61.991537,83.511337,177.705276,505.228301,145.679103,139.021893,238.003087,198.927964,...,473.530760,633.313649,576.277761,585.540324,542.121459,580.890541,613.113028,84.023671,323.866069,591.172152
2,7761872847,1.830102,60.942482,82.462282,176.656221,504.179246,144.630048,137.972838,236.954032,197.878909,...,472.481705,632.264594,575.228706,584.491269,541.072404,579.841486,612.063973,82.974616,322.817014,590.123097
3,8604836683,127.855787,69.841801,92.811330,187.005269,501.750641,154.979097,148.321887,234.525427,195.450304,...,482.830754,634.418102,576.297122,585.559684,551.421452,590.190535,622.413022,92.151918,333.166063,590.886269
4,8604829828,133.448610,75.434624,98.404153,192.598092,507.343464,160.571920,153.914710,240.118250,201.043127,...,488.423577,640.010925,581.889945,591.152507,557.014275,595.783358,628.005845,97.744741,338.758886,596.479092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23103,8449983186,488.847199,433.607729,408.413059,391.837899,591.432793,458.440851,353.059804,454.461877,415.386754,...,71.138156,212.273931,270.988599,268.560182,57.462297,159.850823,192.073310,453.163956,169.648040,310.211680
23104,4072935008,494.307499,439.068029,413.873359,397.298199,596.893093,463.901150,358.520104,459.922177,420.847054,...,109.140970,197.683380,256.398048,253.969631,44.892213,145.260272,177.482759,458.624255,175.108340,295.621128
23105,4072935012,494.874763,439.635293,414.440623,397.865463,597.460357,464.468415,359.087368,460.489442,421.414319,...,109.708234,197.669900,256.384568,253.956152,44.878733,145.246793,177.469280,459.191520,175.675604,295.607649
23106,8558304561,511.931168,456.691698,431.497028,414.921868,614.516762,481.524819,376.143773,477.545846,438.470723,...,126.764639,204.611943,263.326611,260.898195,53.781979,152.188835,184.411322,476.247924,192.732009,302.549692


In [59]:
OD.rename(columns={'Unnamed: 0': 'NN'}, inplace=True) # Repeat for each OD set, if needed.

#### Find first, second, and third nearest destination for each origin node. 

In [61]:
fail_value = 999999999

In [62]:
# Since we only have one destination set, renaming the OD object for quick find/replace on this section of code.
OD_HDurban = OD

# Nearest
OD_HDurban["HDurban1"] = 0
sub = OD_HDurban.iloc[:,1:-1] # Filtering out the newly created field and the node ID column. ("include everything between column 0 and the last column")
OD_HDurban["HDurban1"] = sub.min(axis=1) # Default is axis=0, meaning min value of each column selected. We want min of each row.
HDurban1 = OD_HDurban[['NN', 'HDurban1']] # Remove unnecessary OD values.
HDurban1.to_csv(os.path.join(out_pth, 'fromagriculture/HDurban1_flood.csv'))


# Second nearest
dupes = OD_HDurban.apply(pd.Series.duplicated, axis = 1, keep=False) # If a number is repeated within a row, value is True. If not, False.
# The first time this is done, there should be two True values per row, unless any POIs are equidistant.
dupes = OD_HDurban.where(~dupes, fail_value) # For any value that appears more than once in its row, it is replaced with the fail_value.

OD_HDurban["HDurban2"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column. No need to filter 1st nearest as its new "dupes" value is too high to be caught.
OD_HDurban["HDurban2"] = Dsub.min(axis=1) 
HDurban2 = OD_HDurban.loc[:,['NN', 'HDurban2']] 
HDurban2.to_csv(os.path.join(out_pth, 'fromagriculture/HDurban2_flood.csv'))


# Third nearest
dupes = OD_HDurban.apply(pd.Series.duplicated, axis = 1, keep=False)
# Since this includes both first and second nearest columns, there should be four True values per row, unless POIs are equidistant.
dupes = OD_HDurban.where(~dupes, fail_value)
 
OD_HDurban["HDurban3"] = 0
Dsub = dupes.iloc[:,1:] # Filtering out the node ID column.
OD_HDurban["HDurban3"] = Dsub.min(axis=1)
HDurban3 = OD_HDurban.loc[:,['NN', 'HDurban3']]
HDurban3.to_csv(os.path.join(out_pth, 'fromagriculture/HDurban3_flood.csv'))


# Combine and write to file
HDurban_all = OD_HDurban.loc[:,['NN', 'HDurban1', 'HDurban2', 'HDurban3']]
HDurban_all.to_csv(os.path.join(out_pth, 'fromagriculture/HDurban_all_flood.csv'))
HDurban_all.head()

,NN,HDurban1,HDurban2,HDurban3
0,7761872879,3.601602,4.610503,6.780075
1,7761872860,2.879157,3.888058,6.057630
2,7761872847,1.830102,2.839003,5.008575
3,8604836683,68.078912,69.841801,72.851560
4,8604829828,73.671735,75.434624,78.444383


### Join back to georeferenced _snap file.

In [63]:
ag_snap = os.path.join(out_pth, "ag_snap.csv")
ag_snap = pd.read_csv(ag_snap)
ag_snap

,Unnamed: 0,Unnamed_ 0,mgrs_code,type,GlobalID,Shape_Leng,Shape_Area,geometry,NN,NN_dist
0,0,0,28PCU1265_01,hamlet,{ED2CCDD5-C78F-40B6-A18A-3A01B61A4998},0.004314,0.000001,POINT (-16.721473282454415 12.348636090165247),7761872879,396.520984
1,1,1,28PCU1365_01,hamlet,{372B104B-B208-4D14-84E2-8ABFD4D8C37A},0.009910,0.000006,POINT (-16.716456507935607 12.34788723564788),7761872860,918.665696
2,2,2,28PCU1365_02,hamlet,{D03C2B85-5F35-4EE8-8346-B83494628F26},0.003754,0.000001,POINT (-16.713855008830738 12.350880992129111),7761872847,952.759215
3,3,3,28PCU1566_01,hamlet,{5EAFF1C3-6EE5-4F96-99FC-78F924454480},0.004401,0.000002,POINT (-16.701275174874546 12.355585269999269),7761872847,1923.779554
4,4,4,28PCU1566_02,hamlet,{1D6A9E17-0D49-446D-A23B-7A47B155DC64},0.005357,0.000002,POINT (-16.698773736706396 12.356804484409668),7761872847,2181.254121
...,...,...,...,...,...,...,...,...,...,...
125881,125881,125881,28QED6412_03,hamlet,{5555A010-36B2-47D2-96C4-BDD1E59111ED},0.005397,0.000002,POINT (-14.397827933065358 16.394142941310925),8592243241,5089.086102
125882,125882,125882,28QED6413_03,hamlet,{20205A44-8B9D-4FCE-B14C-53826594DB5A},0.003610,0.000001,POINT (-14.397473236932905 16.405676938062538),8592243457,4062.119008
125883,125883,125883,28QED6413_04,hamlet,{AC6A169C-FD0E-4DF6-BDAB-B69FBD04BFAF},0.015471,0.000008,POINT (-14.400364192239715 16.404303134272737),8592243457,4008.851333
125884,125884,125884,28QED6424_03,hamlet,{51593C65-B268-4BA1-8212-E43232C021FF},0.003883,0.000001,POINT (-14.396243884215123 16.49715066208162),3646207611,1826.174300


In [64]:
ag_snap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125886 entries, 0 to 125885
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  125886 non-null  int64  
 1   Unnamed_ 0  125886 non-null  int64  
 2   mgrs_code   125886 non-null  object 
 3   type        125886 non-null  object 
 4   GlobalID    125886 non-null  object 
 5   Shape_Leng  125886 non-null  float64
 6   Shape_Area  125886 non-null  float64
 7   geometry    125886 non-null  object 
 8   NN          125886 non-null  int64  
 9   NN_dist     125886 non-null  float64
dtypes: float64(3), int64(3), object(4)
memory usage: 9.6+ MB


In [66]:
ag_to_HDurban = pd.merge(ag_snap, HDurban_all, on='NN',how='left')
ag_to_HDurban

,Unnamed: 0,Unnamed_ 0,mgrs_code,type,GlobalID,Shape_Leng,Shape_Area,geometry,NN,NN_dist,HDurban1,HDurban2,HDurban3
0,0,0,28PCU1265_01,hamlet,{ED2CCDD5-C78F-40B6-A18A-3A01B61A4998},0.004314,0.000001,POINT (-16.721473282454415 12.348636090165247),7761872879,396.520984,3.601602,4.610503,6.780075
1,1,1,28PCU1365_01,hamlet,{372B104B-B208-4D14-84E2-8ABFD4D8C37A},0.009910,0.000006,POINT (-16.716456507935607 12.34788723564788),7761872860,918.665696,2.879157,3.888058,6.057630
2,2,2,28PCU1365_02,hamlet,{D03C2B85-5F35-4EE8-8346-B83494628F26},0.003754,0.000001,POINT (-16.713855008830738 12.350880992129111),7761872847,952.759215,1.830102,2.839003,5.008575
3,3,3,28PCU1566_01,hamlet,{5EAFF1C3-6EE5-4F96-99FC-78F924454480},0.004401,0.000002,POINT (-16.701275174874546 12.355585269999269),7761872847,1923.779554,1.830102,2.839003,5.008575
4,4,4,28PCU1566_02,hamlet,{1D6A9E17-0D49-446D-A23B-7A47B155DC64},0.005357,0.000002,POINT (-16.698773736706396 12.356804484409668),7761872847,2181.254121,1.830102,2.839003,5.008575
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125881,125881,125881,28QED6412_03,hamlet,{5555A010-36B2-47D2-96C4-BDD1E59111ED},0.005397,0.000002,POINT (-14.397827933065358 16.394142941310925),8592243241,5089.086102,6.893470,13.243532,14.366722
125882,125882,125882,28QED6413_03,hamlet,{20205A44-8B9D-4FCE-B14C-53826594DB5A},0.003610,0.000001,POINT (-14.397473236932905 16.405676938062538),8592243457,4062.119008,6.594712,12.944774,14.067963
125883,125883,125883,28QED6413_04,hamlet,{AC6A169C-FD0E-4DF6-BDAB-B69FBD04BFAF},0.015471,0.000008,POINT (-14.400364192239715 16.404303134272737),8592243457,4008.851333,6.594712,12.944774,14.067963
125884,125884,125884,28QED6424_03,hamlet,{51593C65-B268-4BA1-8212-E43232C021FF},0.003883,0.000001,POINT (-14.396243884215123 16.49715066208162),3646207611,1826.174300,2.801472,15.323852,16.447042


In [67]:
ag_to_HDurban.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125886 entries, 0 to 125885
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  125886 non-null  int64  
 1   Unnamed_ 0  125886 non-null  int64  
 2   mgrs_code   125886 non-null  object 
 3   type        125886 non-null  object 
 4   GlobalID    125886 non-null  object 
 5   Shape_Leng  125886 non-null  float64
 6   Shape_Area  125886 non-null  float64
 7   geometry    125886 non-null  object 
 8   NN          125886 non-null  int64  
 9   NN_dist     125886 non-null  float64
 10  HDurban1    125883 non-null  float64
 11  HDurban2    125886 non-null  float64
 12  HDurban3    125886 non-null  float64
dtypes: float64(6), int64(3), object(4)
memory usage: 13.4+ MB


In [68]:
ag_to_HDurban.to_csv(os.path.join(out_pth, 'fromagriculture/ag_to_HDurban_flood.csv'))

### End of script. Load into QGIS or Arc and visualize at 10 min intervals. 
QML file for symbology in QGIS:
R:\GEOGlobal\Design\symb_traveltimes_10min.qml